# Etap 1.1.1
## Wczytanie surowych danych według roku, tłumaczenie nazw kolumn
## zapis do pliku "Data/Concatenated Features/by_year_features.csv"


# Cel biznesowy

Celem klasteryzacji wybranego zbioru danych finansowych spółek notowanych na Borsa İstanbul (BIST) jest **grupowanie spółek o podobnych profilach finansowych na podstawie ich bilansów rocznych**.

Dzięki segmentacji spółek według cech takich jak aktywa, pasywa, zyski czy wskaźniki finansowe, możliwe będzie:

- **Identyfikowanie wzorców finansowych** charakterystycznych dla różnych sektorów i etapów rozwoju spółek,
- **Wsparcie decyzji inwestycyjnych i analizy ryzyka** poprzez rozróżnienie grup spółek o stabilnych, rosnących lub problematycznych fundamentach finansowych,
- **Ułatwienie analizy porównawczej** i benchmarkingu wśród firm o podobnej wielkości lub profilu działalności,
- **Automatyzacja analizy dużych zbiorów danych finansowych**, które pochodzą z różnorodnych, nieustrukturyzowanych źródeł, w celu ich lepszego wykorzystania w modelach predykcyjnych i badaniach naukowych.



In [1]:
# run only once
#!pip install deep-translator

In [3]:
import pandas as pd
import numpy as np
import os
from glob import glob
from deep_translator import GoogleTranslator


In [4]:
# Ścieżka do folderu z plikami 
folder_path = "Data/Turkish Public Companies Balance Sheets from KAP"

# Lista wszystkich plików .xlsx 
excel_files = glob(os.path.join(folder_path, "*.xlsx"))

# Lista do zebrania przetworzonych DataFrame'ów
df_list = []

for file in excel_files:
    # Wczytaj plik
    df = pd.read_excel(file)
    
    # Pivot: z długiego na szeroki format (zmiana układu danych)
    df_wide = df.pivot_table(index=[], columns='Desc', values='Yıllık', aggfunc='first').reset_index()
    
    # Dodaj kolumnę z nazwą pliku (bez ścieżki)
    df_wide['source_file'] = os.path.basename(file)
    
    # Dodaj do listy
    df_list.append(df_wide)

# Połącz wszystkie w jeden DataFrame
final_df = pd.concat(df_list, ignore_index=True)

# Dodanie kolumn identyfikujących spółkę i rok na podstawie nazwy pliku
final_df['Nazwa'] = final_df['source_file'].str.extract(r'^([^_]+)')[0]
final_df['Rok'] = final_df['source_file'].str.extract(r'_([0-9]{4})')[0]






In [5]:
# Tłumaczymy wszystkie nazwy kolumn z tureckiego na polski
translated_columns = [GoogleTranslator(source='tr', target='pl').translate(col) for col in final_df.columns]
final_df.columns = translated_columns




In [6]:
print("Przetłumaczone kolumny:")
len( translated_columns)
final_df.head()

Przetłumaczone kolumny:


,indeks,(SubTotal),Przepisy długoterminowe,Amortyzacja i straż pożarna,Kapitał partnerstwa głównego,Akcje mniejszościowe,Przepisy zadłużenia,Żywe istoty,Zachęta państwowa i pomoc,Zwiększenie wartości fundusze,...,Zobowiązania długoterminowe,Inne dochody i wydatki przed opodatkowaniem,Yab. Money Trans. Różnica. Efekt. Poprzedni.nak.ve nak. Tak jak. Wzrost/spadek netto,Sprzedaż zagraniczna,Sprzedaż krajowa,Akcje zysków/strat inwestycji cenianych metodą samowystarczalności,Samo -właściwość,Źródło_pliku,Nazwa,Rok
0,Yıllık,4.524640e+08,678404.0,1242248.0,1.387926e+08,0.0,2852756.0,0.0,0.0,0.0,...,2806817.0,449296.0,-9990337.0,0.0,0.0,0.0,1.387926e+08,A1CAP_2020.xlsx,A1CAP,2020
1,Yıllık,8.515088e+08,0.0,3263754.0,2.966258e+08,0.0,6670834.0,0.0,0.0,0.0,...,8695590.0,777230.0,121976765.0,0.0,0.0,0.0,2.966258e+08,A1CAP_2021.xlsx,A1CAP,2021
2,Yıllık,4.019261e+09,4748296.0,12651941.0,1.487849e+09,0.0,70752251.0,0.0,0.0,0.0,...,132279622.0,17107575.0,463136619.0,0.0,0.0,0.0,1.487849e+09,A1CAP_2022.xlsx,A1CAP,2022
3,Yıllık,6.297396e+09,7715324.0,21452328.0,3.696944e+09,0.0,42331251.0,0.0,0.0,0.0,...,34025698.0,6055765.0,691863980.0,0.0,0.0,0.0,3.696944e+09,A1CAP_2023.xlsx,A1CAP,2023
4,Yıllık,8.297508e+09,9773361.0,31826374.0,4.065838e+09,0.0,28062814.0,0.0,0.0,0.0,...,160295553.0,5169826.0,369461172.0,0.0,0.0,0.0,4.065838e+09,A1CAP_2024.xlsx,A1CAP,2024


In [7]:
# Usunięcie kolumn, w których jest >=40% wartości NaN
threshold = 0.4
nan_ratio = final_df.isna().mean()
columns_to_drop = nan_ratio[nan_ratio >= threshold].index
print(f"\n❌ Kolumny usunięte, ponieważ zawierają >= {int(threshold*100)}% braków danych:")
print(list(columns_to_drop))




❌ Kolumny usunięte, ponieważ zawierają >= 40% braków danych:
['Odponowane przychody (Word Klient Word. Dogan Load. Zewnętrzna. Kal.)', 'Odkładane przychody (Word Client Word.doğan Yür.', 'Prawo do używania aktywów', 'Obietnica klienta. Obciążenie Dogan.', 'Słowo klienta.', 'Aktywa wynikające z umów o klienta', 'Zysk/strata aktywności przed kosztem finansowania']


In [8]:
final_df.drop(columns=columns_to_drop, inplace=True)

In [9]:
# Usunięcie wierszy, gdzie jest >=40% NaN
row_nan_ratio = final_df.isna().mean(axis=1)
rows_to_drop = row_nan_ratio[row_nan_ratio >= threshold].index
print(f"\n❌ Usunięto {len(rows_to_drop)} wierszy z >= {int(threshold*100)}% braków danych")
final_df.drop(index=rows_to_drop, inplace=True)




❌ Usunięto 23 wierszy z >= 40% braków danych


In [10]:
# Zapis do pliku CSV gotowego do dalszej analizy
output_path = "Data/Concatenated Features/by_year_features.csv"
final_df.to_csv(output_path, index=False, encoding='utf-8-sig')